In [1]:
import requests
import pandas as pd
import json

In [77]:
def getAPIData(
    username = 'imunizacao_public',
    password = 'qlto5t&7r_@+#Tlstigi',
    url_firstpage = 'https://imunizacao-es.saude.gov.br/_search?scroll=1m',
    url_nextpages = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    ):

    '''
    Função para extrair todo os dados de vacinação da API.
    Para mais detalhes em relação à execução da chamada da API confira o manual contido dentro da pasta others.
    Retorna o dataframe completo mais o status das suas respectivas respostas de consulta.
    Consulta pode demorar bastante, tempo estimado de 10 páginas por minuto, dependendo do processador.
    '''

    responses = []
    scroll_ids = []

    # Chamada da primeira página.
    response_pag1 = requests.post(url_firstpage, auth=(username, password), json={"size": 10000})
    
    print(f'Page 1 - Status Response: {response_pag1.status_code}')

    responses.append(response_pag1.status_code)

    # Normalização do arquivo em json e tranferência para um Dataframe.
    data = response_pag1.json()
    df = pd.json_normalize(data, record_path=['hits','hits'])

    # Para as próximas chamadas devemos gerar o scroll id para inserir no corpo da chamada da API
    scroll_id = data['_scroll_id']

    items = df.shape[0]

    # Loop while para conferir os conteúdos das demais páginas onde pag representa o número da página
    pag = 1
    while items == 10000 and pag < 100:

        pag+=1

        try:
            # Chamada das demais páginas.
            response_pagN = requests.post(url_nextpages, auth=(username, password), json={"scroll_id":scroll_id,"scroll": "1m"})
    
            if pag%10 == 0:
                print(f'Page {pag} - Status Response: {response_pagN.status_code}')
    
            responses.append(response_pagN.status_code)
    
            # Normalização do arquivo em json e tranferência para um Dataframe.
            dataN = response_pagN.json()
            dfN = pd.json_normalize(dataN, record_path=['hits','hits'])
    
            # Atribuição ao novo valor de items para verificação se a página contém mais do que o número máximo de registros (10000)
            items = dfN.shape[0]
    
            # Concatenação dos dados extraídos das seguintes leituras da API
            df = pd.concat([df,dfN],ignore_index=True)

        except MemoryError:
            print(f'At Page {pag} there was not enough memory to execute the task - Status Response: {response_pagN.status_code}\nThe scroll_id has been added to the list to a future attempt')
            scroll_ids.append(scroll_id)
        
        scroll_id = dataN['_scroll_id']


    print(f'{pag} Pages have been loaded successfully!')

    print(f'The following responses status have been retrieved: {set(responses)}')

    print(f'Final Dataset Shape:\nRows: {df.shape[0]}\nColumns: {df.shape[1]}')
    
    return df, responses, scroll_ids
        

In [73]:
df, responses, scroll_ids = getAPIData()

Page 1 - Status Response: 200
Page 10 - Status Response: 200
Page 20 - Status Response: 200
Page 30 - Status Response: 200
Page 40 - Status Response: 200
Page 50 - Status Response: 200
Page 60 - Status Response: 200
Page 70 - Status Response: 200
Page 80 - Status Response: 200
Page 90 - Status Response: 200
Page 100 - Status Response: 200
101 Pages have been loaded successfully!
The following responses status have been retrieved: {200}
Final Dataset Shape:
Rows: 1010000
Columns: 46


In [76]:
df.head(2)

,_index,_type,_id,_score,_source.estalecimento_noFantasia,_source.vacina_lote,_source.estabelecimento_municipio_codigo,_source.estabelecimento_valor,_source.vacina_nome,_source.ds_condicao_maternal,...,_source.paciente_endereco_nmMunicipio,_source.estabelecimento_municipio_nome,_source.vacina_codigo,_source.paciente_enumSexoBiologico,_source.dt_deleted,_source.co_condicao_maternal,_source.vacina_grupoAtendimento_nome,_source.paciente_racaCor_valor,_source.vacina_dataAplicacao,_source.data_importacao_rnds
0,desc-imunizacao-v5,_doc,46450ed3-85a9-4660-a182-e961f0fc1dc7-i0b0,1.0,UBS DR DANILO APARECIDO DE OLIVEIRA,222VCD048Z,354020,9720901,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,NaN,...,PONTAL,PONTAL,85,M,2022-09-08T00:00:00.000Z,NaN,Pessoas de 18 a 64 anos,PRETA,2022-09-08T00:00:00.000Z,2022-09-08T14:09:04.000Z
1,desc-imunizacao-v5,_doc,a13bfb49-6537-4654-bad7-460409bd4729-i0b0,1.0,UBS DOM PEDRO I,222VCD048Z,354990,0009660,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,NaN,...,SAO JOSE DOS CAMPOS,SAO JOSE DOS CAMPOS,85,F,2022-09-08T00:00:00.000Z,NaN,Pessoas de 18 a 64 anos,SEM INFORMACAO,2022-09-08T00:00:00.000Z,2022-09-08T14:09:06.000Z
